# Working with Narrative Data

This notebook illustrates some examples of working with narrative text data using small, local language models.

## Running this notebook on a newer MacBook with Apple Silicon Chip

You will need an environment with Python and Jupyter installed. To create an environment with Anaconda for Python 3.12, execute: 

```
conda create --name llm-narrative python=3.12
conda activate llm-narrative
conda install jupyter
jupyter notebook
```

## Running this notebook on older MacBooks or any other machine

Please run this script on [Google Colab](https://colab.research.google.com/). After opening the notebook there, please change the settings to using a GPU, check [here](https://www.geeksforgeeks.org/how-to-use-gpu-in-google-colab/) for instructions on how to do that.



In [1]:
## check computer platform
import platform
print(platform.processor())

Intel64 Family 6 Model 165 Stepping 2, GenuineIntel


### Install required libraries

For the newer MacBooks with Apple Chips we will use `mlx-lm` to load a small, quantized version of the Llama 3 8b instruct model, so that it can run on a single laptop (https://ollama.com/library/llama3). For older MacBooks and other machines we will use a quantized version of the model provided by the hugging face community (https://huggingface.co/astronomer/Llama-3-8B-Instruct-GPTQ-4-Bit).

Depending on the machine, different packages are required and will be installed below.

In [2]:
import platform

# for the newer MacBooks with the Apple Chip
# changed for testing but change back later
if platform.processor() == 'arm':
    ! pip install mlx-lm torch transformers
# for all other machines
else:
    ! pip install torch transformers optimum accelerate auto-gptq bitsandbytes

### Install Llama 3 - 8b
Next we install the quantized version of the Llama 8b language model.



In [3]:
if platform.processor() == 'arm':
    from mlx_lm import load, generate
    model, tokenizer = load("mlx-community/Meta-Llama-3-8B-Instruct-4bit")
else:
    from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
    import torch

    MODEL_ID="astronomer/Llama-3-8B-Instruct-GPTQ-4-Bit"
    tokenizer = AutoTokenizer.from_pretrained("astronomer/Llama-3-8B-Instruct-GPTQ-4-Bit")

    config = AutoConfig.from_pretrained(MODEL_ID)
    config.quantization_config["disable_exllama"] = False
    config.quantization_config["exllama_config"] = {"version":2}

    model = AutoModelForCausalLM.from_pretrained(
            MODEL_ID, 
            device_map='auto', 
            torch_dtype=torch.bfloat16, 
            trust_remote_code=True, 
            # low_cpu_mem_usage=True,
            # load_in_4bit=True,
            config=config,
        )

OSError: [WinError 126] 找不到指定的模块。 Error loading "E:\Apps\anaconda\envs\rdf\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

### Running the model with an example prompt

We show that the model can run with an example prompt. First we define the system prompt, which tells the model what character to adopt. Then we give it an instruction to introduce itself. Again, depending on the machine and therefore model used, we use slightly different functions to generate output.

In [ ]:
from transformers import pipeline
from IPython.display import display

SYSTEM_MSG = "You are a helpful chatbot assistant."

def generateFromPrompt(promptStr,maxTokens=100):
    if platform.processor() == 'arm':
      messages = [ {"role": "system", "content": SYSTEM_MSG},
              {"role": "user", "content": promptStr}, ]
      input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
      prompt = tokenizer.decode(input_ids)
      response = generate(model, tokenizer, prompt=prompt, max_tokens=maxTokens)
    else:
      message = [{"role": "user", "content": promptStr},]
      pipe = pipeline("text-generation", model=model, tokenizer=tokenizer,max_new_tokens=maxTokens)
      result = pipe(message)
      response = result[0]['generated_text'][1]['content']
    return(response)


response = generateFromPrompt("Please introduce yourself")

print(response+"...")

Now we illustrate performance on a slightly more medically themed topic:

In [ ]:
response = generateFromPrompt("Please tell me some symptoms of allergies",maxTokens=200)

print(response+"...")

### Using the model to extract themes from narrative texts

Here we demonstrate how the model can be used to extract themes from a narrative interview text.

This brief interview extract comes from the article Price, T., McColl, E., & Visram, S. (2022). Barriers and facilitators of childhood flu vaccination: The views of parents in North East England. Zeitschrift Fur Gesundheitswissenschaften = Journal of Public Health, 30(11), 2619–2626. https://doi.org/10.1007/s10389-022-01695-2

```
So can you tell me a kind of your first impressions, your first thoughts around flu? That can be anything from, like, is it a serious illness, what kind of symptoms do you expect to see, kind of anything first thoughts around flu.

Participant 12: Yeah, so I know a little bit from my yeah, kind of studies and from work, etc. I think it is quite a serious issue. I think, you know, moving forward as well with the latest pandemic. It could be worse, given the strains on the healthcare system and things like that.But yeah, I think I'm certainly aware of the the seriousness of it for certain at risk groups in particular. So anyone who's already vulnerable or has a long term condition and who are elderly, orvery young, obviously are at higher risk and yeah, you know, the whole aim of the kind of screening program has been to protect those at risk groups up, you know, first and foremost. Also in terms of developmening, herd immunity across the whole community as well. So I've always been for the campaign obviously I understand the evidence base behind it and I always had a flu jab myself. You know, we get provided with that as healthcare workers and as public health workers through our employer. But equally, I think, you know, we see the benefits to, to our children as well in terms of protecting them. But also protecting all the vulnerable people that they come in contact with as well, obviously, when they're at school, when they see elderly grandparents and things like that as well. So, um, as far as signs and symptoms. I think I possibly only had flu once in my life and it may have just been a really bad bug. But I was literally bed bound for it when I was at university for a few, good few days. And so it may have been may have been flu. I think from a symptoms point of view it was just the usual kind of fever, nausea, muscle aches, just high temperature and things like that, really, yeah. Yeah, so they were probably my, my kind of bad symptoms, I would say.

```



In [ ]:
interviewExtract = """
So can you tell me a kind of your first impressions, your first thoughts around flu? That can be anything from, like, is it a serious illness, what kind of symptoms do you expect to see, kind of anything first thoughts around flu.

Participant 12: Yeah, so I know a little bit from my yeah, kind of studies and from work, etc. I think it is quite a serious issue. I think, you know, moving forward as well with the latest pandemic. It could be worse, given the strains on the healthcare system and things like that.But yeah, I think I'm certainly aware of the the seriousness of it for certain at risk groups in particular. So anyone who's already vulnerable or has a long term condition and who are elderly, orvery young, obviously are at higher risk and yeah, you know, the whole aim of the kind of screening program has been to protect those at risk groups up, you know, first and foremost. Also in terms of developmening, herd immunity across the whole community as well. So I've always been for the campaign obviously I understand the evidence base behind it and I always had a flu jab myself. You know, we get provided with that as healthcare workers and as public health workers through our employer. But equally, I think, you know, we see the benefits to, to our children as well in terms of protecting them. But also protecting all the vulnerable people that they come in contact with as well, obviously, when they're at school, when they see elderly grandparents and things like that as well. So, um, as far as signs and symptoms. I think I possibly only had flu once in my life and it may have just been a really bad bug. But I was literally bed bound for it when I was at university for a few, good few days. And so it may have been may have been flu. I think from a symptoms point of view it was just the usual kind of fever, nausea, muscle aches, just high temperature and things like that, really, yeah. Yeah, so they were probably my, my kind of bad symptoms, I would say.
"""

response = generateFromPrompt("Please identify some themes in the following interview transcript: ' "+interviewExtract+"'",maxTokens=200)

print(response+"...")

### Exploring structural aspects of the interview text

We can also try to use the LM to extract structural aspects of the interview speech such as how confident the speaker is.

In [ ]:
response = generateFromPrompt("Please describe how confident the interviewee is in the following interview, with motivation: ' "+interviewExtract+"'",maxTokens=200)

display(response+"...")

### Exploring emotional aspects of the interview text

We can explore any emotional dimensions that are present in the narrative text.

In [ ]:
response = generateFromPrompt("Please describe the sentiment and any emotions of the interviewee in the following interview, with motivation: ' "+interviewExtract+"'",maxTokens=200)

print(response+"...")

### Extracting information according to a predefined schema

The original study from which this transcript has been sourced harnessed the COM-B theory of behaviour change to frame the research study into perspectives on childhood vaccination. We can similarly use the language model to extract examples of those theoretical elements.

In [ ]:
response = generateFromPrompt("Please extract examples of COM-B (capability, opportunity, and motivation) elements in the following interview: ' "+interviewExtract+"'",maxTokens=400)

print(response+"...")